In [1]:
suppressPackageStartupMessages(library(dplyr))
suppressPackageStartupMessages(library(tidyr))
suppressPackageStartupMessages(library(tibble))
suppressPackageStartupMessages(library(ggplot2))
suppressPackageStartupMessages(library(stringr))
library(ggrepel)
library(reshape2)
library(ggsci)
library(rstatix)
library(ggpubr)
library(data.table)
library(ggtern)
library(viridis)
library(scales)
library(RColorBrewer)
library(parallel)
library(purrr)
source('/home//jupyter/BRI_Figures_Final_V2/helper_function/helper_function_IHA.r')


Attaching package: ‘reshape2’


The following object is masked from ‘package:tidyr’:

    smiths



Attaching package: ‘rstatix’


The following object is masked from ‘package:stats’:

    filter



Attaching package: ‘data.table’


The following objects are masked from ‘package:reshape2’:

    dcast, melt


The following objects are masked from ‘package:dplyr’:

    between, first, last


Registered S3 methods overwritten by 'ggtern':
  method           from   
  grid.draw.ggplot ggplot2
  plot.ggplot      ggplot2
  print.ggplot     ggplot2

--
Remember to cite, run citation(package = 'ggtern') for further info.
--


Attaching package: ‘ggtern’


The following object is masked from ‘package:rstatix’:

    mahalanobis_distance


The following objects are masked from ‘package:ggplot2’:

    aes, annotate, ggplot, ggplot_build, ggplot_gtable, ggplotGrob,
    ggsave, layer_data, theme_bw, theme_classic, theme_dark,
    theme_gray, theme_light, theme_linedraw, theme_minimal, theme_void




In [2]:
meta_data=read.csv("/home/jupyter/BRI_Figures_Final_V2/Figure5/01_Frequency_Comparison/selected_samples_with_acutal_flu_year.csv")

# Get Pathway Scores

In [3]:
# SLEA - assigning pathway scores
doSLEA <- function(expressionSet, geneSet) {
  # scale expression
  exprsMat <- expressionSet
  #exprsMat <- t(scale(t(exprsMat)))
  # extract expression of leGenes of each geneset
  comm <- intersect(geneSet, rownames(expressionSet))
  gsDF <- exprsMat[comm, ]
  # calculate mean expression per sample
  gsM <- colMeans(gsDF)
  # extract random genes of size of the geneSet from full probeset and calculate mean
  # and perform this for 'n' permutations
  nperm <- lapply(1:1000, function(j) {
    # set seed for every permutation
    set.seed(j)
    rGSDF <- exprsMat[sample.int(nrow(exprsMat),length(comm)), ]
    rGSM <- colMeans(rGSDF)
    return(value = rGSM)
  })
  permDF <- do.call(rbind, nperm)
  zscore <- (gsM - colMeans(permDF)) / apply(permDF,2,sd)
  sleaDF <- zscore %>% as.data.frame()
  return(value = sleaDF)
}

In [4]:
meta_data_selected_RNA_subset_D0<-meta_data %>% filter(Flu_Year=="2020-2021",sample.visitName	%in%c("Flu Year 1 Day 7") ) 


In [5]:
file_list<-paste0("/home/jupyter/BRI_Figures_Final_V2/Dataset/scRNA/BRI/Average_LogNormalized_Expression/Average_LogNormalized_Expression_of_Celltypes_by_Sample_AIFI_L3/",meta_data_selected_RNA_subset_D0$pbmc_sample_id,".csv")

df_list<-read_pseudobulk_expression(file_list)


[1] "Total reading time: 9.654 seconds"
[1] "The length of the list matches the length of the input path."


In [6]:
background_gene<-read.csv('/home/jupyter/BRI_Figures_Final_V2/Figure5/02_DEGs_and_EnrichmentAnalysis/filtered_gene_Y2020-2021_Y2021-2022_D7.csv') %>% filter(AIFI_L3=='Core memory B cell')

In [7]:
Selected_Pathways<-c("HALLMARK_TNFA_SIGNALING_VIA_NFKB",
                     'HALLMARK_PI3K_AKT_MTOR_SIGNALING',
                     "HALLMARK_MTORC1_SIGNALING",
                    'HALLMARK_MYC_TARGETS_V1',
                     'HALLMARK_MYC_TARGETS_V2',
                     'HALLMARK_ALLOGRAFT_REJECTION')

In [8]:
res_list<-list()
for (pathway_single in Selected_Pathways){
gene_set<-read.csv('/home/jupyter/BRI_Figures_Final_V2/Figure5/02_DEGs_and_EnrichmentAnalysis/gsea_result_all.csv') %>% 
filter(pathway==pathway_single,
celltype=='Core memory B cell')
gene_vector <- eval(parse(text = gene_set$leadingEdge))

  
celltype_list <- lapply(df_list, function(df) { df <- df[background_gene$gene, grep('Core memory B cell', names(df), fixed = TRUE),drop=FALSE] })


exp_matrix<-do.call(cbind,celltype_list)
colnames(exp_matrix)<-sub(":.*", "", colnames(exp_matrix))
rownames(meta_data)<-meta_data$pbmc_sample_id

res<-doSLEA(exp_matrix,gene_vector)

res$pbmc_sample_id<-rownames(res)
res<-left_join(res,meta_data)

res<-res[c('subject.subjectGuid','.')]
colnames(res)<-c('subject.subjectGuid',paste0("CoreMemoryB_",pathway_single))
res_list[[pathway_single]]<-res
    
}

Joining with `by = join_by(pbmc_sample_id)`
Joining with `by = join_by(pbmc_sample_id)`
Joining with `by = join_by(pbmc_sample_id)`
Joining with `by = join_by(pbmc_sample_id)`
Joining with `by = join_by(pbmc_sample_id)`
Joining with `by = join_by(pbmc_sample_id)`


In [9]:
pathway_variable<- reduce(res_list, full_join, by = "subject.subjectGuid")

# HAI assay phuket Delta changes

In [11]:
df<-read.csv("/home//jupyter/BRI_Figures_Final_V2/Extended_Figure5_1//05_HAI/FC_phuket.csv")

In [12]:
df<-df[c('subject.subjectGuid','fold_change')]

In [13]:
colnames(df)<-c(c('subject.subjectGuid','HAI_fold_change'))

In [14]:
HAI_variable<-df

# MSD assay phuket Fold Changes

In [15]:
df<-read.csv('/home//jupyter/BRI_Figures_Final_V2/Figure5/04_MSD/MSD_Normalized_Y2020-2021_Phuket.csv')

In [16]:
df<-df %>% filter(Visit=='Flu Year 1 Day 7')

In [17]:
df<-df[c('subject.subjectGuid','Normalized_Concentration')]

In [18]:
colnames(df)<-c(c('subject.subjectGuid','MSD_phuket_fold_change'))

In [19]:
MSD_variable<-df

# Frequency Changes

In [20]:
df<-read.csv('/home//jupyter/BRI_Figures_Final_V2/Figure5/01_Frequency_Comparison/freq_actual_flu_year_1_D0_D7.csv')

In [21]:
df_CoreMemoryB<-df %>% filter(Flu_Year=='2020-2021',AIFI_L3=='Core memory B cell') %>%  
select(subject.subjectGuid,AIFI_L3_clr,Flu_Day)%>%
  dplyr::group_by(subject.subjectGuid) %>%
  tidyr::spread(Flu_Day, AIFI_L3_clr) %>%
  dplyr::mutate( CoreMemoryB_Freq_Fold_Changes = `Day 7` - `Day 0`) %>% 
  select(subject.subjectGuid,CoreMemoryB_Freq_Fold_Changes)

df_Plasma<-df %>% filter(Flu_Year=='2020-2021',AIFI_L3=='Plasma cell') %>%  
select(subject.subjectGuid,AIFI_L3_clr,Flu_Day)%>%
  dplyr::group_by(subject.subjectGuid) %>%
  tidyr::spread(Flu_Day, AIFI_L3_clr) %>%
  dplyr::mutate( Plasma_Freq_Fold_Changes = `Day 7` - `Day 0`)%>% 
  select(subject.subjectGuid,Plasma_Freq_Fold_Changes)

In [22]:
scRNA_Fold_Change_variable<-left_join(df_Plasma,df_CoreMemoryB)

Joining with `by = join_by(subject.subjectGuid)`


# Mean Expression

In [23]:
meta_data_selected_RNA_subset_D7<-meta_data %>% filter(Flu_Year=="2020-2021",sample.visitName	%in%c("Flu Year 1 Day 7") ) 


In [24]:
aggregated_count_file_list<-paste0("/home/jupyter/BRI_Figures_Final_V2/Dataset/scRNA/BRI/Average_LogNormalized_Expression/Average_LogNormalized_Expression_of_Celltypes_by_Sample_AIFI_L3/",meta_data_selected_RNA_subset_D0$pbmc_sample_id,".csv")
df_list<-read_pseudobulk_expression(aggregated_count_file_list)

[1] "Total reading time: 4.889 seconds"
[1] "The length of the list matches the length of the input path."


In [25]:
selected_genes<-c("IGHG3","CD19")


In [26]:
df<-filter_genes_and_celltype(df_list, selected_genes, c('CD27- effector B cell'), longformat = TRUE)


Warning message:
“The melt generic in data.table has been passed a data.frame and will attempt to redirect to the relevant reshape2 method; please note that reshape2 is superseded and is no longer actively developed, and this redirection is now deprecated. To continue using melt methods from reshape2 while both libraries are attached, e.g. melt.list, you can prepend the namespace, i.e. reshape2::melt(exp_matrix). In the next version, this warning will become an error.”


In [27]:
df<-left_join(df,meta_data_selected_RNA_subset_D7)

Joining with `by = join_by(pbmc_sample_id)`


In [28]:
expression_IGHG3<-df %>% filter(Gene=="IGHG3") %>% 
select(subject.subjectGuid,Mean_Expression)%>% as.data.frame()

expression_CD19<-df %>% filter(Gene=="CD19") %>% 
select(subject.subjectGuid,Mean_Expression)%>% as.data.frame()

In [29]:
colnames(expression_IGHG3)<-c("subject.subjectGuid","Mean_Expression_IGHG3")
colnames(expression_CD19)<-c("subject.subjectGuid","Mean_Expression_CD19")

In [30]:
mean_expression_variable<-reduce(list(expression_IGHG3,expression_CD19), full_join, by = "subject.subjectGuid")

# Combine them together

In [31]:
combined_df <- reduce(list(pathway_variable,
                           MSD_variable,
                          HAI_variable,
                          scRNA_Fold_Change_variable,mean_expression_variable), 
                      full_join, 
                      by = "subject.subjectGuid")

In [32]:
write.csv(combined_df,"BRI_Bcell_data_forcorrelation.csv")